In [35]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import io
import base64

# Function to generate a summary based on pitcher data
def generate_pitcher_summary(player_data):
    avg_velocity = player_data['RelSpeed'].mean()
    avg_spin = player_data['SpinRate'].mean()
    return f"Average Velocity: {avg_velocity:.1f} mph | Average Spin Rate: {avg_spin:.1f} rpm"


# Initialize the app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    # File upload component
    html.Label("Upload a CSV File with Pitcher Data"),
    dcc.Upload(
        id='upload-data',
        children=html.Button("Upload File"),
        multiple=False
    ),
    
    # Dropdown for selecting the player
    html.Label("Select Pitcher"),
    dcc.Dropdown(id='pitcher-dropdown'),

    # RadioItems or Dropdown for selecting pitch type (filtered by pitcher selection)
    html.Label("Select Pitch Type"),
    dcc.RadioItems(id='pitch-type-radio', value='All'),

    # Placeholder for the pitcher summary text
    html.Div(id='pitcher-summary'),

    # Two side-by-side graphs
    html.Div([
        dcc.Graph(id='pitch-metrics-graph'),
        dcc.Graph(id='pitch-location-graph')
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    
     # Two additional graphs for lefty and righty pitches
    html.Div([
        dcc.Graph(id='pitch-location-lefty-graph'),
        dcc.Graph(id='pitch-location-righty-graph')
    ], style={'display': 'flex', 'justify-content': 'space-between'})
])

# Callback to process the uploaded file and set pitcher dropdown options
@app.callback(
    Output('pitcher-dropdown', 'options'),
    Input('upload-data', 'contents')
)
def load_file(contents):
    if contents is None:
        return []

    # Decode and read the CSV file
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    data = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    
    # Generate dropdown options for pitchers
    pitcher_options = [{'label': name, 'value': name} for name in data['Pitcher'].unique()]
    
    # Save the data globally or in the session
    app.data = data  # Store data globally for use in other callbacks
    
    return pitcher_options

# Callback to update pitch type options based on selected pitcher
@app.callback(
    Output('pitch-type-radio', 'options'),
    Input('pitcher-dropdown', 'value')
)
def update_pitch_type_options(selected_pitcher):
    if selected_pitcher is None or not hasattr(app, 'data'):
        return []
    
    # Filter data for the selected pitcher
    filtered_data = app.data[app.data['Pitcher'] == selected_pitcher]
    
    # Get unique pitch types for the selected pitcher
    pitch_types = [{'label': pitch, 'value': pitch} for pitch in filtered_data['TaggedPitchType'].unique()]
    
    # Add an "All" option to show all pitches
    pitch_types.append({'label': 'All', 'value': 'All'})
    
    return pitch_types

# Callback to update both graphs and summary based on dropdown selections
@app.callback(
    Output('pitch-metrics-graph', 'figure'),
    Output('pitch-location-graph', 'figure'),
    Output('pitch-location-lefty-graph', 'figure'),
    Output('pitch-location-righty-graph', 'figure'),
    Output('pitcher-summary', 'children'),
    Input('pitcher-dropdown', 'value'),
    Input('pitch-type-radio', 'value')
)
def update_graphs(selected_pitcher, selected_pitch_type):
    # Ensure data is available
    if not hasattr(app, 'data'):
        return {}, {}, {}, {}, "Please upload a file first."

    # Filter data for the selected pitcher and pitch type
    filtered_data = app.data[app.data['Pitcher'] == selected_pitcher]
    if selected_pitch_type != 'All':
        filtered_data = filtered_data[filtered_data['TaggedPitchType'] == selected_pitch_type]

    # Create the pitch metrics plot (e.g., velocity vs. spin rate)
    metrics_fig = px.scatter(filtered_data, x='RelSpeed', y='SpinRate', color='TaggedPitchType',
                             title=f"{selected_pitcher}'s Pitches (Velocity vs Spin Rate)")

    # Create the pitch location plot (using x and y coordinates)
    location_fig = px.scatter(filtered_data, x='PlateLocSide', y='PlateLocHeight', color='TaggedPitchType',
                              title=f"{selected_pitcher}'s Pitch Locations", 
                              labels={'PlateLocSide': 'Horizontal Location', 'PlateLocHeight': 'Vertical Location'})
    location_fig.update_xaxes(range=[-3, 3])
    location_fig.update_yaxes(range=[0, 5])
    location_fig.update_layout(
        yaxis=dict(scaleanchor="x", scaleratio=1)  # This ensures 1:1 aspect ratio
    )
    
    # Add the strike zone
    location_fig.add_shape(type="rect", x0=-0.708, x1=0.708, y0=1.5, y1=3.5, line=dict(color="black"))
    
    # Add home plate (pitcher’s perspective)
    location_fig.add_shape(type="line", x0=-0.708, y0=0, x1=0.708, y1=0, line=dict(color="black"))  # Top edge
    location_fig.add_shape(type="line", x0=-0.708, y0=0, x1=-0.708, y1=0.3, line=dict(color="black"))  # Left edge
    location_fig.add_shape(type="line", x0=0.708, y0=0, x1=0.708, y1=0.3, line=dict(color="black"))    # Right edge
    location_fig.add_shape(type="line", x0=-0.708, y0=0.3, x1=0, y1=0.6, line=dict(color="black"))     # Left diagonal
    location_fig.add_shape(type="line", x0=0.708, y0=0.3, x1=0, y1=0.6, line=dict(color="black"))      # Right diagonal

    # Add attack zones (outer heart, inner shadow, etc.)
    location_fig.add_shape(type="rect", x0=-0.558, x1=0.558, y0=1.833, y1=3.166, line=dict(color="purple", width=3))
    location_fig.add_shape(type="rect", x0=-1.108, x1=1.108, y0=1.166, y1=3.833, line=dict(color="orange", width=3))
    location_fig.add_shape(type="rect", x0=-1.666, x1=1.666, y0=0.5, y1=4.5, line=dict(color="grey", width=3))
    
      # Filter data for left-handed batters and create lefty location plot
    lefty_data = filtered_data[filtered_data['BatterSide'] == 'Left']
    location_lefty_fig = px.scatter(lefty_data, x='PlateLocSide', y='PlateLocHeight', color='TaggedPitchType',
                                    title=f"{selected_pitcher}'s Pitch Locations (Lefty Batters)",
                                    labels={'PlateLocSide': 'Horizontal Location', 'PlateLocHeight': 'Vertical Location'})
    location_lefty_fig.update_xaxes(range=[-2, 2])
    location_lefty_fig.update_yaxes(range=[0, 5])
    location_lefty_fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1))

     # Add the strike zone
    location_lefty_fig.add_shape(type="rect", x0=-0.708, x1=0.708, y0=1.5, y1=3.5, line=dict(color="black"))
    
    # Add home plate (pitcher’s perspective)
    location_lefty_fig.add_shape(type="line", x0=-0.708, y0=0, x1=0.708, y1=0, line=dict(color="black"))  # Top edge
    location_lefty_fig.add_shape(type="line", x0=-0.708, y0=0, x1=-0.708, y1=0.3, line=dict(color="black"))  # Left edge
    location_lefty_fig.add_shape(type="line", x0=0.708, y0=0, x1=0.708, y1=0.3, line=dict(color="black"))    # Right edge
    location_lefty_fig.add_shape(type="line", x0=-0.708, y0=0.3, x1=0, y1=0.6, line=dict(color="black"))     # Left diagonal
    location_lefty_fig.add_shape(type="line", x0=0.708, y0=0.3, x1=0, y1=0.6, line=dict(color="black"))      # Right diagonal

    # Add attack zones (outer heart, inner shadow, etc.)
    location_lefty_fig.add_shape(type="rect", x0=-0.558, x1=0.558, y0=1.833, y1=3.166, line=dict(color="purple", width=3))
    location_lefty_fig.add_shape(type="rect", x0=-1.108, x1=1.108, y0=1.166, y1=3.833, line=dict(color="orange", width=3))
    location_lefty_fig.add_shape(type="rect", x0=-1.666, x1=1.666, y0=0.5, y1=4.5, line=dict(color="grey", width=3))
    
    # Filter data for right-handed batters and create righty location plot
    righty_data = filtered_data[filtered_data['BatterSide'] == 'Right']
    location_righty_fig = px.scatter(righty_data, x='PlateLocSide', y='PlateLocHeight', color='TaggedPitchType',
                                     title=f"{selected_pitcher}'s Pitch Locations (Righty Batters)",
                                     labels={'PlateLocSide': 'Horizontal Location', 'PlateLocHeight': 'Vertical Location'})
    location_righty_fig.update_xaxes(range=[-2, 2])
    location_righty_fig.update_yaxes(range=[0, 5])
    location_righty_fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1))
    
       # Add the strike zone
    location_righty_fig.add_shape(type="rect", x0=-0.708, x1=0.708, y0=1.5, y1=3.5, line=dict(color="black"))
    
    # Add home plate (pitcher’s perspective)
    location_righty_fig.add_shape(type="line", x0=-0.708, y0=0, x1=0.708, y1=0, line=dict(color="black"))  # Top edge
    location_righty_fig.add_shape(type="line", x0=-0.708, y0=0, x1=-0.708, y1=0.3, line=dict(color="black"))  # Left edge
    location_righty_fig.add_shape(type="line", x0=0.708, y0=0, x1=0.708, y1=0.3, line=dict(color="black"))    # Right edge
    location_righty_fig.add_shape(type="line", x0=-0.708, y0=0.3, x1=0, y1=0.6, line=dict(color="black"))     # Left diagonal
    location_righty_fig.add_shape(type="line", x0=0.708, y0=0.3, x1=0, y1=0.6, line=dict(color="black"))      # Right diagonal

    # Add attack zones (outer heart, inner shadow, etc.)
    location_righty_fig.add_shape(type="rect", x0=-0.558, x1=0.558, y0=1.833, y1=3.166, line=dict(color="purple", width=3))
    location_righty_fig.add_shape(type="rect", x0=-1.108, x1=1.108, y0=1.166, y1=3.833, line=dict(color="orange", width=3))
    location_righty_fig.add_shape(type="rect", x0=-1.666, x1=1.666, y0=0.5, y1=4.5, line=dict(color="grey", width=3))

    
    # Generate summary
    summary = generate_pitcher_summary(filtered_data)
    
    return metrics_fig, location_fig, location_lefty_fig, location_righty_fig, summary

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
